In [ ]:
import os  # 운영 체제와 상호작용하기 위한 모듈
import shutil  # 파일 및 디렉토리 작업을 위한 모듈
import random  # 랜덤 작업을 위한 모듈
import scipy  # 과학 및 기술 계산을 위한 모듈
import numpy as np  # 수치 연산을 위한 Python 라이브러리
import matplotlib.pyplot as plt  # 데이터 시각화를 위한 라이브러리
import tensorflow as tf  # 텐서플로우 라이브러리
import keras  # 케라스 라이브러리
from keras.preprocessing.image import ImageDataGenerator  # 이미지 데이터 생성기 유틸리티
from keras.models import Model  # Keras의 함수형 API 모델
from keras.layers import (Conv2D, DepthwiseConv2D, LayerNormalization, Dense, GlobalAveragePooling2D, 
                          Input, Layer, BatchNormalization, Activation, Softmax)  # 신경망의 레이어 구성 요소
from keras.optimizers import Adam  # Adam 최적화 알고리즘
from tensorflow_addons.optimizers import AdamW  # AdamW 최적화 알고리즘
from keras.losses import CategoricalCrossentropy  # 손실 함수
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau  # 학습 중 콜백 함수

# 데이터 생성기 설정 함수
def generators(train_dir, val_dir, size=64, image_size=64):
    """학습 및 검증 데이터를 생성하는 함수"""
    # 학습 데이터 생성기 설정
    train_datagen = ImageDataGenerator(rescale=1/255,  # 이미지의 픽셀 값을 0-1 범위로 정규화
                                       rotation_range=180,  # 최대 180도 회전
                                       width_shift_range=0.2,  # 가로 방향 이동
                                       height_shift_range=0.2,  # 세로 방향 이동
                                       shear_range=0.2,  # 시어 변환
                                       zoom_range=0.2,  # 줌 인/줌 아웃
                                       horizontal_flip=True,  # 가로 방향 뒤집기
                                       vertical_flip=True,  # 세로 방향 뒤집기
                                       fill_mode='nearest')  # 변환 중 생기는 빈 공간을 주변의 유사한 픽셀 값으로 채우기

    # 검증 데이터 생성기 설정
    val_datagen = ImageDataGenerator(rescale=1/255)  # 이미지의 픽셀 값을 0-1 범위로 정규화

    # 학습 데이터 생성
    train_generator = train_datagen.flow_from_directory(train_dir,  # 학습 데이터가 위치한 디렉토리
                                                        target_size=(image_size, image_size),  # 입력 이미지 크기
                                                        batch_size=size,  # 배치 크기
                                                        class_mode='categorical',  # 다중 클래스 분류
                                                        shuffle=True)  # 데이터를 섞어서 배치 생성

    # 검증 데이터 생성
    val_generator = val_datagen.flow_from_directory(val_dir,  # 검증 데이터가 위치한 디렉토리
                                                    target_size=(image_size, image_size),  # 입력 이미지 크기
                                                    batch_size=size,  # 배치 크기
                                                    class_mode='categorical')  # 다중 클래스 분류

    return train_generator, val_generator  # 학습 및 검증 데이터 생성기 반환

# 학습 및 검증 데이터 디렉토리 설정
train_dir = 'your_path'
val_dir = 'your_path'

# 데이터 생성기 생성
train_generator, val_generator = generators(train_dir, val_dir)

# Local Perception Unit (LPU) 클래스 정의
class LocalPerceptionUnit(Layer):
    def __init__(self, filters):
        super(LocalPerceptionUnit, self).__init__()
        self.dw_conv = DepthwiseConv2D(kernel_size=3, padding='same')
        self.filters = filters

    def call(self, x):
        return x + self.dw_conv(x)
    
    def get_config(self):
        config = super().get_config()
        config.update({"filters": self.filters})
        return config

# Lightweight Multi-Head Self-Attention (MHSA) 클래스 정의
class LightweightMHSA(Layer):
    def __init__(self, dim, num_heads=8):
        super(LightweightMHSA, self).__init__()
        self.num_heads = num_heads
        self.dim = dim

    def build(self, input_shape):
        self.qkv = Dense(self.dim * 3)
        self.proj = Dense(self.dim)

    def call(self, x):
        B, H, W, C = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2], self.dim
        N = H * W  # Flattened spatial dimensions
        qkv = self.qkv(x)
        qkv = tf.reshape(qkv, (B, N, 3, self.num_heads, C // self.num_heads))
        qkv = tf.transpose(qkv, perm=[2, 0, 3, 1, 4])
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = tf.nn.softmax(tf.matmul(q, k, transpose_b=True) / (C // self.num_heads) ** 0.5, axis=-1)
        attn = tf.matmul(attn, v)

        attn = tf.transpose(attn, perm=[0, 2, 1, 3])
        attn = tf.reshape(attn, (B, H, W, C))

        return self.proj(attn)
    
    def get_config(self):
        config = super().get_config()
        config.update({"dim": self.dim, "num_heads": self.num_heads})
        return config

# Inverted Residual Feed-Forward Network (FFN) 클래스 정의
class InvertedResidualFFN(Layer):
    def __init__(self, dim, expansion_ratio=4):
        super(InvertedResidualFFN, self).__init__()
        self.hidden_dim = int(dim * expansion_ratio)
        self.expand = Dense(self.hidden_dim)
        self.depthwise = DepthwiseConv2D(kernel_size=3, padding='same')
        self.project = Dense(dim)
        self.bn1 = BatchNormalization()
        self.bn2 = BatchNormalization()
        self.gelu = Activation('gelu')

    def call(self, x):
        residual = x
        x = self.expand(x)
        x = self.bn1(x)
        x = self.gelu(x)
        x = self.depthwise(x)
        x = self.bn2(x)
        x = self.project(x)
        return residual + x
    
    def get_config(self):
        config = super().get_config()
        config.update({"dim": self.hidden_dim // 4, "expansion_ratio": 4})
        return config

# CMTBlock (Convolutional Multi-head Self-Attention Block) 클래스 정의
class CMTBlock(Layer):
    def __init__(self, dim, num_heads=8, mlp_ratio=4.):
        super(CMTBlock, self).__init__()
        self.lpu = LocalPerceptionUnit(dim)
        self.lmhsa = LightweightMHSA(dim, num_heads)
        self.irffn = InvertedResidualFFN(dim, mlp_ratio)
        self.norm1 = LayerNormalization()
        self.norm2 = LayerNormalization()

    def call(self, x):
        x = self.lpu(x)
        x = x + self.lmhsa(self.norm1(x))
        x = x + self.irffn(self.norm2(x))
        return x
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "dim": self.lpu.filters,
            "num_heads": self.lmhsa.num_heads,
            "mlp_ratio": self.irffn.hidden_dim // self.lpu.filters
        })
        return config

# CMT-S 모델 정의 함수
def cmt_s(input_shape=(224, 224, 3), num_classes=1000):
    inputs = Input(shape=input_shape)
    
    # Stem
    x = Conv2D(32, kernel_size=3, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('gelu')(x)
    x = Conv2D(32, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('gelu')(x)
    x = Conv2D(32, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('gelu')(x)
    
    # Stage 1
    x = Conv2D(64, kernel_size=2, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('gelu')(x)
    for _ in range(3):
        x = CMTBlock(64)(x)
        
    # Stage 2
    x = Conv2D(128, kernel_size=2, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('gelu')(x)
    for _ in range(3):
        x = CMTBlock(128)(x)
    
    # Stage 3
    x = Conv2D(256, kernel_size=2, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('gelu')(x)
    for _ in range(16):
        x = CMTBlock(256)(x)
    
    # Stage 4
    x = Conv2D(512, kernel_size=2, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('gelu')(x)
    for _ in range(3):
        x = CMTBlock(512)(x)
    
    x = GlobalAveragePooling2D()(x)
    
    x = Dense(1280)(x)
    x = BatchNormalization()(x)
    x = Activation('gelu')(x)
    
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    return model

# 모델 생성 및 컴파일
model = cmt_s(input_shape=(64, 64, 3), num_classes=100)
model.compile(optimizer=Adam(learning_rate=1e-4, decay=5e-9), loss=CategoricalCrossentropy(from_logits=False), metrics=['accuracy'])

# 모델 요약
model.summary()

# 모델 체크포인트 및 조기 종료 콜백 설정
model_path = 'your_path'
CP = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)
ES = EarlyStopping(monitor='val_loss', patience=10)

# 모델 학습
history = model.fit(train_generator, epochs=100, validation_data=val_generator, batch_size=16, callbacks=[CP, ES])

# 학습 결과 시각화
plt.plot(history.history['loss'])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['val_accuracy'])
plt.title('CMT_S Model')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'accuracy', 'val_loss', 'val_accuracy'], loc='upper right')
plt.show()

# 최적의 검증 정확도와 손실 출력
print('Best Val Acc:', max(history.history['val_accuracy']))
print('Best Val Loss:', min(history.history['val_loss']))